In [6]:
import ast

# Your dictionary
my_dict = {"50": "120+1j", 150: "200.0", 250: 300+200j, "no_num": 300, 50: "no_num", 100: {"real":1, "imag":100}}

def is_number(value):
    """Check if the value is a number or a numerical string."""
    if isinstance(value, (int, float, complex)):
        return True
    elif isinstance(value, str):
        try:
            # Safely evaluate the string to check if it's a number
            result = ast.literal_eval(value)
            return isinstance(result, (int, float, complex))
        except (ValueError, SyntaxError):
            return False
    elif isinstance(value, dict):
        # Check if dict represents a complex number with 'real' and 'imag' keys
        if set(value.keys()) == {'real', 'imag'}:
            real = value['real']
            imag = value['imag']
            # Check if both 'real' and 'imag' are numbers
            return isinstance(real, (int, float)) and isinstance(imag, (int, float))
        else:
            return False
    else:
        return False

# Iterate over the dictionary and check keys and values
for key, val in my_dict.items():
    key_is_number = is_number(key)
    val_is_number = is_number(val)
    print(f"Key: {key} (numerical: {key_is_number}), Value: {val} (numerical: {val_is_number})")


Key: 50 (numerical: True), Value: 120+1j (numerical: True)
Key: 150 (numerical: True), Value: 200.0 (numerical: True)
Key: 250 (numerical: True), Value: (300+200j) (numerical: True)
Key: no_num (numerical: False), Value: 300 (numerical: True)
Key: 50 (numerical: True), Value: no_num (numerical: False)
Key: 100 (numerical: True), Value: {'real': 1, 'imag': 100} (numerical: True)


In [6]:
import sympy as sp

#function which returns the coefficents

def get_coefficents(eq, var:list):
    """Return the coefficients of the given equation for the given variable."""
    # Get the polynomial representation of the equation, iterate over all var terms which are given as a list
    poly = eq.as_poly(var)

    coeffs = [sp.simplify(poly.coeff_monomial(i)) for i in var]
    coeffs.append(poly.coeff_monomial(1))
    # Return the coefficients
    return coeffs

def get_par(U_list:list, Y_list:list):
    #sum up all the elements of the list
    Yp = 0
    Ip = 0
    for U, Y in zip(U_list, Y_list):
        Yp = Yp + Y
        Ip = Ip + U*Y
    Up = Ip/Yp
    return Up, Yp


#solving the equations of a basic model 

Y11, Y22, Y33, Y44, Y55, Y66, Y77 = sp.symbols("Y_11, Y_22, Y_33, Y_44, Y_55, Y_66, Y_77")
Y12, Y23, Y34, Y35, Y56, Y67, Y71, Y36 = sp.symbols("Y_12, Y_23, Y_34, Y_35, Y_56, Y_67, Y_71, Y_36")
Y12a, Y12b = sp.symbols("Y_12a, Y_12b")
Y56a, Y56b, Y56c = sp.symbols("Y_56a, Y_56b, Y_56c")
IS1, IS2, IS3 = sp.symbols("I_S1, I_S2, I_S3")
U12a, U12b, U23, U34, U35, U56a, U56b, U56c, U67, U71, U36 = sp.symbols("U_12a, U_12b, U_23, U_34, U_35, U_56a, U_56b, U_56c, U_67, U_71, U_36")
uk1, uk2, uk3, uk4, uk5, uk6, uk7 = sp.symbols("u_k1, u_k2, u_k3, u_k4, u_k5, u_k6, u_k7")


#solition for nodes node 1
U12p, Y12p = get_par([U12a, U12b], [Y12a, Y12b])
I12 = Y12p * (uk2 - uk1) + Y12p*U12p
U71p, Y71p = get_par([U71], [Y71])
I71 = (Y71p * (uk7 - uk1) + Y71p * U71p)
eq1 = sp.Eq(IS1 - I71 + uk1*Y11 - I12, 0)

#solution for node 2
U23p, Y23p = get_par([U23], [Y23])
I23 = Y23p * (uk3 - uk2) + Y23p * U23p
eq2 = sp.Eq(I12 + uk2*Y22 - I23, 0)

#solution for node 3
U34p, Y34p = get_par([U34], [Y34])
U35p, Y35p = get_par([U35], [Y35])
U36p, Y36p = get_par([U36], [Y36])
I34 = Y34p * (uk4 - uk3) + Y34p * U34p
I35 = Y35p * (uk5 - uk3) + Y35p * U35p
I36 = Y36p * (uk6 - uk3) + Y36p * U36p
eq3 = sp.Eq(I23 + uk3*Y33 - I35 - I34 - I36, 0)

#solution for node 4
eq4 = sp.Eq(I34 + IS2 +uk4*Y44, 0)

#solution for node 5
U56p, Y56p = get_par([U56a, U56b, U56c], [Y56a, Y56b, Y56c])
I56 = Y56p * (uk6 - uk5) + Y56p * U56p
IF = IS1 + IS2 + IS3
eq5 = sp.Eq(I35 +Y55*uk5 - I56 - IF, 0)

#solution for node 6
U67p, Y67p = get_par([U67], [Y67])
I67 = Y67p * (uk7 - uk6) + Y67p * U67p
eq6 = sp.Eq(I56 + uk6*Y66 - I67, 0)

#solution for node 7
eq7 = sp.Eq(I67 + uk7*Y77 + IS3 + I71, 0)

coeffs1 = get_coefficents(eq1, [uk1, uk2, uk3, uk4, uk5, uk6, uk7])
coeffs2 = get_coefficents(eq2, [uk1, uk2, uk3, uk4, uk5, uk6, uk7])
coeffs3 = get_coefficents(eq3, [uk1, uk2, uk3, uk4, uk5, uk6, uk7])
coeffs4 = get_coefficents(eq4, [uk1, uk2, uk3, uk4, uk5, uk6, uk7])
coeffs5 = get_coefficents(eq5, [uk1, uk2, uk3, uk4, uk5, uk6, uk7])
coeffs6 = get_coefficents(eq6, [uk1, uk2, uk3, uk4, uk5, uk6, uk7])
coeffs7 = get_coefficents(eq7, [uk1, uk2, uk3, uk4, uk5, uk6, uk7])


coeffs6



[0,
 0,
 0,
 0,
 -Y_56a - Y_56b - Y_56c,
 Y_56a + Y_56b + Y_56c + Y_66 + Y_67,
 -Y_67,
 U_56a*Y_56a + U_56b*Y_56b + U_56c*Y_56c - U_67*Y_67]

In [2]:
import sympy as sp

valid_formula = "2*x + 3*y - 4*z + 5"
invalid_formula = "2*x + 3*y - 4*z + 5 +"

sp.simplify(valid_formula)
sp.simplify(invalid_formula)

SympifyError: Sympify of expression 'could not parse '2*x + 3*y - 4*z + 5 +'' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 1)